# 02b - Vertex AI - AutoML with clients (code)
Use the Vertex AI Python Client to recreate the no-code approach of (02a) with code (Python).  This builds a custom model with AutoML and deploys it to an Endpoint for predictions and explanations.  

**Prerequisites:**
-  01 - BigQuery - Table Data Source

**Overview:**
-  Use Python client google.cloud.aiplatform for Vertex AI
   -  Create a dataset
      -  aiplatform.TabularDataset
      -  Link BigQuery table
   -  Train Model with AutoML
      -  aiplatform.AutoMLTabularTrainingJob
   -  Evaluate
      -  Review the model in GCP Console > Vertex AI > Models
   -  Deploy to Endpoint
      -  Endpoint = aiplatform.Endpoint
      -  Endpoint.deploy
   -  Online Predictions
      -  Endpoint.predict
   -  Explanations
      -  Endpoint.explain
   -  Batch Prediction Job
      -  aiplatform.BatchPredictionJob

**Resources:**
-  [Python Client for Vertex AI](https://googleapis.dev/python/aiplatform/latest/aiplatform.html)
-  [AutoML Tabular Training Job With Python Client](https://cloud.google.com/vertex-ai/docs/training/automl-api#aiplatform_create_training_pipeline_tabular_classification_sample-python)
-  [Interpreting Explanations](https://cloud.google.com/vertex-ai/docs/predictions/interpreting-results-automl#tabular)

**Related Training:**
-  todo


---
## Conceptual Architecture

<img src="architectures/statmike-mlops-A2.png">

---
## Setup

inputs:

In [1]:
REGION = 'us-central1'
PROJECT_ID='statmike-mlops'
DATANAME = 'digits'
NOTEBOOK = '02b'

# Resources
DEPLOY_COMPUTE = 'n1-standard-4'

packages:

In [2]:
from google.cloud import aiplatform
from datetime import datetime

from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

clients:

In [3]:
aiplatform.init(project=PROJECT_ID, location=REGION)

parameters:

In [4]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

---
## Create Dataset (link to BigQuery table)

In [5]:
dataset = aiplatform.TabularDataset.create(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}', 
    bq_source = f'bq://{PROJECT_ID}.{DATANAME}.{DATANAME}_prepped',
    labels = {'notebook':f'{NOTEBOOK}'}
)

INFO:google.cloud.aiplatform.datasets.dataset:Creating TabularDataset
INFO:google.cloud.aiplatform.datasets.dataset:Create TabularDataset backing LRO: projects/691911073727/locations/us-central1/datasets/496266772219953152/operations/7184871706522025984
INFO:google.cloud.aiplatform.datasets.dataset:TabularDataset created. Resource name: projects/691911073727/locations/us-central1/datasets/496266772219953152
INFO:google.cloud.aiplatform.datasets.dataset:To use this TabularDataset in another session:
INFO:google.cloud.aiplatform.datasets.dataset:ds = aiplatform.TabularDataset('projects/691911073727/locations/us-central1/datasets/496266772219953152')


---
## Train Model with AutoML

The feature columns are named `p0` through `p63`. Using this naming, the column_specs dictionary of datatypes can be defined:

In [6]:
column_specs = {}
for i in range(64):
    column_specs['p'+str(i)] = 'auto'

In [7]:
tabular_classification_job = aiplatform.AutoMLTabularTrainingJob(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    optimization_prediction_type = 'classification',
    column_specs = column_specs,
    labels = {'notebook':f'{NOTEBOOK}'}
)

In [8]:
model = tabular_classification_job.run(
    dataset = dataset,
    target_column = 'target',
    predefined_split_column_name = 'splits',
    budget_milli_node_hours = 1000,
    model_display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    disable_early_stopping = False,
    model_labels = {'notebook':f'{NOTEBOOK}'}
)

INFO:google.cloud.aiplatform.training_jobs:View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/8307648175708045312?project=691911073727
INFO:google.cloud.aiplatform.training_jobs:AutoMLTabularTrainingJob projects/691911073727/locations/us-central1/trainingPipelines/8307648175708045312 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:AutoMLTabularTrainingJob projects/691911073727/locations/us-central1/trainingPipelines/8307648175708045312 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:AutoMLTabularTrainingJob projects/691911073727/locations/us-central1/trainingPipelines/8307648175708045312 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:AutoMLTabularTrainingJob projects/691911073727/locations/us-central1/trainingPipelines/8307648175708045312 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud

---
## Endpoint and Deployment

In [45]:
endpoint = aiplatform.Endpoint.create(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    labels = {'notebook':f'{NOTEBOOK}'}
)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/691911073727/locations/us-central1/endpoints/8262117399201841152/operations/8906240722689064960
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/691911073727/locations/us-central1/endpoints/8262117399201841152
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/691911073727/locations/us-central1/endpoints/8262117399201841152')


In [46]:
endpoint.deploy(
    model = model,
    deployed_model_display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    traffic_percentage = 100,
    machine_type = DEPLOY_COMPUTE,
    min_replica_count = 1,
    max_replica_count = 1
)

INFO:google.cloud.aiplatform.models:Deploying Model projects/691911073727/locations/us-central1/models/1466686938802028544 to Endpoint : projects/691911073727/locations/us-central1/endpoints/8262117399201841152
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/691911073727/locations/us-central1/endpoints/8262117399201841152/operations/4744914666998726656
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/691911073727/locations/us-central1/endpoints/8262117399201841152


---
## Prediction

### Prepare a record for prediction: instance and parameters lists

In [48]:
%%bigquery pred
SELECT *
FROM `digits.digits`
LIMIT 10

Downloading: 100%|██████████| 10/10 [00:01<00:00,  9.97rows/s]


In [49]:
pred.head(4)

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,p56,p57,p58,p59,p60,p61,p62,p63,target,target_OE
0,0.0,5.0,16.0,15.0,5.0,0.0,0.0,0.0,0.0,2.0,...,0.0,6.0,16.0,16.0,16.0,16.0,7.0,0.0,2,Even
1,0.0,5.0,16.0,12.0,1.0,0.0,0.0,0.0,0.0,5.0,...,0.0,8.0,16.0,16.0,16.0,16.0,4.0,0.0,2,Even
2,0.0,5.0,15.0,16.0,6.0,0.0,0.0,0.0,0.0,11.0,...,0.0,6.0,16.0,16.0,16.0,13.0,3.0,0.0,2,Even
3,0.0,4.0,15.0,15.0,8.0,0.0,0.0,0.0,0.0,8.0,...,0.0,7.0,14.0,11.0,0.0,0.0,0.0,0.0,2,Even


In [50]:
newob = pred.loc[:0,'p0':'p63'].to_dict(orient='records')[0]
#newob

In [51]:
instances = [json_format.ParseDict(newob, Value())]
parameters = json_format.ParseDict({}, Value())

### Prediction: Online

In [52]:
prediction = endpoint.predict(instances=instances, parameters=parameters)

In [53]:
prediction

Prediction(predictions=[{'classes': ['4', '0', '5', '3', '7', '6', '9', '1', '8', '2'], 'scores': [9.944386351705414e-14, 1.024329384334555e-12, 3.424070194046891e-10, 2.287049660765206e-08, 1.203022403917231e-10, 1.640781538414515e-10, 4.202649513118661e-12, 9.056180516608947e-08, 6.825113700870133e-07, 0.9999992251396179]}], deployed_model_id='6765611705054527488', explanations=None)

In [54]:
import numpy as np
prediction.predictions[0]['classes'][np.argmax(prediction.predictions[0]['scores'])]

'2'

## Explanation
Interpretation Guide
- https://cloud.google.com/vertex-ai/docs/predictions/interpreting-results-automl#tabular

In [55]:
explanation = endpoint.explain(instances=instances, parameters=parameters)

In [56]:
explanation.predictions

[{'scores': [9.944386351705414e-14,
   1.024329384334555e-12,
   3.424070194046891e-10,
   2.287049660765206e-08,
   1.203022403917231e-10,
   1.640781538414515e-10,
   4.202649513118661e-12,
   9.056180516608947e-08,
   6.825113700870133e-07,
   0.9999992251396179],
  'classes': ['4', '0', '5', '3', '7', '6', '9', '1', '8', '2']}]

In [57]:
print("attribution:")
print("baseline output",explanation.explanations[0].attributions[0].baseline_output_value)
print("instance output",explanation.explanations[0].attributions[0].instance_output_value)
print("output_index",explanation.explanations[0].attributions[0].output_index)
print("output display value",explanation.explanations[0].attributions[0].output_display_name)
print("approximation error",explanation.explanations[0].attributions[0].approximation_error)

attribution:
baseline output 0.0007730582146905363
instance output 0.9999992251396179
output_index [9]
output display value 2
approximation error 0.025125523680607203


In [58]:
explanation.explanations[0].attributions[0]

baseline_output_value: 0.0007730582146905363
instance_output_value: 0.9999992251396179
feature_attributions {
  struct_value {
    fields {
      key: "p0"
      value {
        number_value: 0.0
      }
    }
    fields {
      key: "p1"
      value {
        number_value: 0.02430733598469357
      }
    }
    fields {
      key: "p10"
      value {
        number_value: 0.0
      }
    }
    fields {
      key: "p11"
      value {
        number_value: 0.002543250996885555
      }
    }
    fields {
      key: "p12"
      value {
        number_value: 0.00788054584492264
      }
    }
    fields {
      key: "p13"
      value {
        number_value: 0.06045667270414664
      }
    }
    fields {
      key: "p14"
      value {
        number_value: 0.0
      }
    }
    fields {
      key: "p15"
      value {
        number_value: 0.0
      }
    }
    fields {
      key: "p16"
      value {
        number_value: 0.0
      }
    }
    fields {
      key: "p17"
      value {
        nu

### Batch Predictions: BigQuery Source to BigQuery Destination, with Explanations

In [59]:
batch = aiplatform.BatchPredictionJob.create(
    job_display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    model_name = model.name,
    instances_format = "bigquery",
    predictions_format = "bigquery",
    bigquery_source = f'bq://{PROJECT_ID}.{DATANAME}.{DATANAME}',
    bigquery_destination_prefix = f"{PROJECT_ID}",
    generate_explanation = True,
    labels = {'notebook':f'{NOTEBOOK}'}
)

INFO:google.cloud.aiplatform.jobs:Creating BatchPredictionJob
INFO:google.cloud.aiplatform.jobs:BatchPredictionJob created. Resource name: projects/691911073727/locations/us-central1/batchPredictionJobs/7908657396220690432
INFO:google.cloud.aiplatform.jobs:To use this BatchPredictionJob in another session:
INFO:google.cloud.aiplatform.jobs:bpj = aiplatform.BatchPredictionJob('projects/691911073727/locations/us-central1/batchPredictionJobs/7908657396220690432')
INFO:google.cloud.aiplatform.jobs:View Batch Prediction Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/7908657396220690432?project=691911073727
INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/691911073727/locations/us-central1/batchPredictionJobs/7908657396220690432 current state:
JobState.JOB_STATE_RUNNING
INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/691911073727/locations/us-central1/batchPredictionJobs/7908657396220690432 current state:
JobState.JOB_STAT